<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso1/ciclo3/3_ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1e7ctPi8O3bTQoLZaO9ZZjwGr2r8Z93RS" width="100%">

# N-grams
---

En este notebook veremos cómo podemos extraer características a partir de texto utilizando una estrategia de representación conocida como bolsas de N-grams (BoN - Bag-of-N-grams). Primero importamos las librerías necesarias:

In [ ]:
!pip install unidecode

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, IFrame
from sklearn.feature_extraction.text import CountVectorizer
from unidecode import unidecode
plt.style.use("ggplot")

## **1. Motivación y Definición**
---

Los N-grams son un tipo de representación que generaliza las bolsas de palabras (BoW), de hecho, un BoW corresponde a un caso específico de unigrama (1-gram a nivel de palabra).

Este tipo de *embedding* consiste en conteos de secuencias de tokens, es decir, se busca una distribución de secuencias $s_j$ dado un determinado documento $d_i$, de esta forma: $P(S=s_j | D=d_i)$.

Los N-grams se definen como secuencias de tokens de longitud $N$, veamos un ejemplo con la palabra:

> `"universidad"`:

- Los 1-grams a nivel de carácter son: `"u"`, `"n"`, `"i"`, `"v"`, `"e"`, `"r"`, `"s"`, `"d"`, `"a"`.
- Los 2-grams a nivel de carácter son: `"un"`, `"ni"`, `"iv"`, "`ve`", `"er"`, `"rs"`, `"si"`, `"id"`, `"da"`, `"ad"`.
- Los 3-grams a nivel de carácter son: `"uni"`, `"niv"`, `"ive"`, `"ver"`, `"ers"`, `"rsi"`, `"sid"`, `"ida"`, `"dad"`.

También es posible calcular N-grams a nivel palabra. Veamos un ejemplo con la oración:

> `"Albert Einstein era un científico"`

- Los 1-grams a nivel de palabra son: `"Albert"`, `"Einstein"`, `"era"`, `"un"`, `"científico"`.
- Los 2-grams a nivel de palabra son: `"Albert Einstein"`, `"Einstein era"`, `"era un"`, `"un científico"`.
- Los 3-grams a nivel de palabra son: `"Albert Einstein era"`, `"Einstein era un"`, `"era un científico"`.

En la siguiente figura se presenta un ejemplo con N-grams a nivel de palabra para una frase más larga:

<img src="https://drive.google.com/uc?export=view&id=16YxKp76Uu9SzbIn1RwKYq0IBmPoXHe29" width="100%">


Veamos un ejemplo en _Python_ para calcular los N-grams. Primero definimos un texto de ejemplo:

In [ ]:
text = "este es un texto de ejemplo para calcular n-grams y sus conteos"

Primero, calculamos todos los posibles 2-grams:

In [ ]:
bigrams = list(set(text[i: i + 2] for i in range(len(text) - 2)))
display(bigrams)

Ahora, podemos calcular los conteos de bigramas en el texto:

In [ ]:
counts = {bigram: text.count(bigram) for bigram in bigrams}
display(counts)

Veamos un diagrama de barras de los conteos de bigramas:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax.bar(counts.keys(), counts.values())
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.set_xlabel("Bigrama")
ax.set_ylabel("Conteo")
fig.show()

## **2. Implementación con sklearn**
---

Utilizando las clases `CountVectorizer` y `TfidfVectorizer` de `sklearn` podemos calcular representaciones de N-grams de la misma forma en la que calculábamos bolsas de palabras, para ello, haremos uso de los siguientes dos parámetros:

- `analyzer`: específica si se calculan n-grams a nivel `"char"` (carácter) o `"word"` (palabra).
- `ngram_range`: específica el rango de grams a considerar, por ejemplo, `(1, 3)` calcula 1-grams, 2-grams y 3-grams.

Veamos un ejemplo sobre el siguiente corpus:

In [ ]:
corpus = [
        "albert einstein era un científico",
        "la teoría de la relatividad de albert einstein"
        ]

Veamos un ejemplo de bigramas a nivel de palabra:

In [ ]:
vect = CountVectorizer(
        analyzer="word",
        ngram_range=(2, 2)
        ).fit(corpus)

Extraemos la representación y  el vocabulario:

In [ ]:
X = vect.transform(corpus).toarray()
vocab  = vect.get_feature_names_out()

Veamos la representación como un `DataFrame`:

In [ ]:
X_pd = pd.DataFrame(columns=vocab, data=X)
display(X_pd)

Veamos otro ejemplo con trigramas a nivel de carácter:

In [ ]:
vect = CountVectorizer(
        analyzer="char",
        ngram_range=(3, 3)
        ).fit(corpus)

Extraemos la representación y  el vocabulario:

In [ ]:
X = vect.transform(corpus).toarray()
vocab  = vect.get_feature_names_out()

Veamos la representación como un `DataFrame`:

In [ ]:
X_pd = pd.DataFrame(columns=vocab, data=X)
display(X_pd)

## **3. Modelos Probabilísticos del Lenguaje**
---

Los modelos probabilísticos del lenguaje son una forma clásica de entender los textos en procesamiento de lenguaje natural, estos modelos parten de una distribución conjunta de determinado documento $d_i$ compuesto de un conjunto $T=\{t_1,t_2,\dots, t_l\}$ de términos ordenados:

$$
P(t_1, t_2, \dots, t_l | d_i) = P(t_1 | d_i) P(t_2 | t_1, d_i) P(t_3 | t_1, t_2, d_i) \dots P(t_l | t_{l-1}, t_{l-2}, \dots, t_1, d_i)
$$

Por ejemplo, la distribución conjunta del siguiente ejemplo $d_0$ = `"Esta es la casa que Jack construyó"` sería:

$$
\begin{align}
P(\text{Esta, es, la, casa, que, Jack, construyó}) &= P(\text{Esta})P(\text{es}|\text{Esta})P(\text{la} | \text{Esta}, \text{es})\\
&~~~~ \dots P(\text{construyó} | \text{Esta,es,la,casa,que,Jack})
\end{align}
$$

Los N-grams surgen naturalmente al suponer que los documentos son [cadenas de Markov](https://es.wikipedia.org/wiki/Cadena_de_M%C3%A1rkov). Más específicamente, una suposición de Markov de grado $K$ considera un $(K+1)$-gram. Las _cadenas de Markov_ son modelos probabilísticos en los que se asume un tipo específico de independencia condicional, en el caso de texto, una cadena de grado $K$ representa que cada palabra es condicionada únicamente por sus $K$ palabras precedentes. Por ejemplo, veamos la siguiente cadena de grado 1:

<img src="https://drive.google.com/uc?export=view&id=1Q-ORveyrHZy8gt_cqJ4xcwGioXfrmgdh" width="80%">

Se puede ver que cada palabra depende únicamente de su predecesora, y palabras como `"Photography"`, `"Science"` y `"Mathematics"` son condicionalmente independientes de `"I"`, es decir:

$$
P(\text{Photography} | \text{like, I}) = P(\text{Photography} | \text{like})
$$

Adicionalmente, bajo esta independencia condicional, sólo se forman distribuciones entre secuencias de dos palabras, es decir, un 2-gram.

Veamos algunos ejemplos de aplicación de los modelos probabilísticos del lenguaje.

### **3.1. Autocompletado**
---

El objetivo del autocompletado es determinar la secuencia de caracteres o palabras más probable en una distribución condicional.

In [ ]:
#@markdown ##**Ejecute esta celda para ver el video.**
IFrame(
        src="https://drive.google.com/file/d/19fQ6RGRKZ-dCIv6Ypoku7n0PsaQj5J_W/preview",
        width="768px",
        height="432px"
        )

Por ejemplo, en el siguiente corpus:

- Esta es la casa que Jack construyó.
- Esta es la casa de Pedro.
- Esta es la tienda.
- Esta es la rata.
- Esta es la gata.

Si partimos de la secuencia `"Esta es la"` podemos construir la distribución: $P(t_4 | t_1 = Esta, t_2 = es, t_3 = la)$. La cual puede ser usada para determinar la palabra que sigue de acuerdo a la siguiente regla:

$$
w_4 = \text{argmax}_{t_i} P(t_4 | t_1 = Esta, t_2 = es, t_3 = la)
$$

En este caso, sabemos que la palabra más probable es `"casa"`, ya que tenemos las siguientes probabilidades:

- $P(\text{casa} | \text{Esta, es la}) = \frac{2}{5}$.
- $P(\text{tienda} | \text{Esta, es la}) = \frac{1}{5}$.
- $P(\text{rata} | \text{Esta, es la}) = \frac{1}{5}$.
- $P(\text{gata} | \text{Esta, es la}) = \frac{1}{5}$.

Veamos un ejemplo práctico sobre el dataset [TMDB](https://www.themoviedb.org/?language=en-US) con resúmenes de películas:

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/mindlab-unal/mlds4-datasets/main/u3/movies.csv")
display(data)

En específico el corpus está contenido en la columna `"overview"`:

In [ ]:
corpus = (
        data
        .dropna(subset=["overview"])
        .overview.tolist()
        )
display(corpus)

Vamos a definir una función de preprocesamiento para el corpus:

In [ ]:
pat = re.compile(r"[^a-z ]")
spaces = re.compile(r"\s{2,}")
def preprocess(text, min_len=1, max_len=23):
    # Normalizamos el texto
    norm_text = unidecode(text).lower()

    # Extraemos tokens
    tokens = norm_text.split()

    # Filtramos palabras por longitud
    filtered_tokens = filter(
            lambda token: len(token) >= min_len and len(token) <= max_len,
            tokens
        )
    filtered_text = " ".join(filtered_tokens)
    # Eliminamos caracteres especiales
    clean_text = re.sub(pat, "", filtered_text)
    # Eliminamos espacios duplicados
    spaces_text = re.sub(spaces, " ", clean_text)
    return spaces_text.strip()

Obtenemos el corpus preprocesado:

In [ ]:
corpus_prep = list(map(preprocess, corpus))
display(corpus_prep)

Finalmente, calculamos una representación de 3-grams a nivel de palabra:

In [ ]:
vect = CountVectorizer(
        ngram_range = (3, 3),
        analyzer = "word"
        )
counts = np.array(vect.fit_transform(corpus_prep).sum(axis=0)).flatten()
display(counts)

También, extraemos una lista de todos los 3-grams encontrados:

In [ ]:
grams = list(
        map(
            lambda i: i.split(" "),
            vect.get_feature_names_out()
            )
        )
display(grams)

Como puede ver, tenemos secuencias de 3 palabras y el conteo de cada secuencia, podemos construir un `DataFrame` con esta información y los conteos:

In [ ]:
grams_df = pd.DataFrame(
        grams, columns=[f"t{i}" for i in range(1, 4)]
        ).assign(counts=counts)
display(grams_df)

En este caso, podemos autocompletar tomando secuencias de dos tokens (tenemos 3-grams), veamos un ejemplo con la siguiente secuencia:

In [ ]:
sent = ["accused", "of"]

Puede probar otros ejemplos de autocompletado con las siguientes secuencias:

In [ ]:
# sent = ["accompanied","by"]
# sent = ["about", "to"]
# sent = ["about", "the"]
# sent = ["be", "an"]

Vamos a filtrar todos los trigramas donde los primeros dos términos coinciden con la secuencia que deseamos autocompletar:

In [ ]:
valid_grams = grams_df.query("t1 == @sent[0] and t2 == @sent[1]")
display(valid_grams)

Como podemos ver, en la columna `"t3"` tenemos las posibles palabras que ocurren en el corpus luego de `"accused of"`, podemos normalizar estos valores para que sean probabilidades válidas:

In [ ]:
valid_grams = valid_grams.assign(
        prob = valid_grams.counts / valid_grams.counts.sum()
        )
display(valid_grams)

Podemos ver las probabilidades como un diagrama de barras:

In [ ]:
fig, ax = plt.subplots()
ax.bar(valid_grams.t3, valid_grams.prob)
ax.set_xlabel("$t_3$")
ax.set_ylabel("$P(t_3 | t_1, t_2)$")
fig.show()

Finalmente, podemos generar la palabra autocompletada como el término con mayor probabilidad:

In [ ]:
word = valid_grams.iloc[valid_grams.prob.argmax()].t3
display(word)

### **3.2. Modelos Generativos**
---

La generación automática de texto a partir del modelo probabilístico de N-grams utiliza las probabilidades condicionales que vimos anteriormente y técnicas de generación de números aleatorios. En este caso, utilizaremos el generador de números aleatorios de `numpy` para generar palabras de forma iterativa.

Podemos generar automáticamente secuencias de caracteres, de acuerdo a la siguiente ecuación:

$$
t_{i+2} \sim P(t_{i+2} | t_{i+1}, t_{i})
$$

Donde $t_{i + 2}$ es un token generado a partir de la secuencia $t_i, ~t_{i + 1}$.

In [ ]:
#@markdown ##**Ejecute esta celda para ver el video.**
IFrame(
        src="https://drive.google.com/file/d/1XTEbe7WElxWfaVouuNXZSUtvBhljzCY6/preview",
        width="768px",
        height="432px"
        )

Veamos un ejemplo de texto generado usando el modelo de trigramas que teníamos definido, para ello, comenzaremos con una secuencia de dos palabras:

In [ ]:
seq = ["about", "to"]

También definimos un número de palabras a generar:

In [ ]:
N = 100

Iterativamente calculamos probabilidades y generamos nuevas palabras:

> **Nota**: puede ejecutar el siguiente código varias veces y en cada una el resultado será distinto, ya que es un modelo generativo.

In [ ]:
for i in range(N):
    sent = seq[-2:] # seleccionamos las últimas 2 palabras.
    probs = (
            grams_df
            .query("t1 == @sent[0] and t2 == @sent[1]") # filtramos coincidencias
            .assign(prob = lambda df: df.counts / df.counts.sum()) # calculamos las probabilidades
            .filter(["t3", "prob"]) # filtramos las columnas en cuestión
            )
    if probs.shape[0] == 0: # cuando un n-gram no está en el vocabulario
        probs = (
                grams_df
                .query("t1 == 'this' and t2 == 'is'") # completamos con un 2-gram muy común
                .assign(prob = lambda df: df.counts / df.counts.sum())
                .filter(["t3", "prob"])
                )
    word = np.random.choice(probs.t3, p=probs.prob) # seleccionamos una palabra
    seq.append(word)

Veamos el resultado:

In [ ]:
display(" ".join(seq))

En este caso, obtuvimos un texto que sigue reglas del lenguaje, pero no tiene mucha coherencia. Esto es un problema típico en las representaciones de N-grams y se conoce como el problema de la memoria a corto plazo, la representación de trigramas que estamos usando únicamente considera las últimas dos palabras para generar una tercera, es decir, el modelo olvida por completo dependencias a largo plazo con otras palabras.

Hoy en día se suelen usar modelos de redes neuronales (como redes recurrentes o transformers) que solucionan el problema de la memoria a corto plazo. Este tipo de modelos se estudiarán en el módulo de _Deep Learning_.

## **4. Entendimiento de Idiomas**
---

Una de las aplicaciones típicas de los N-grams es para el entendimiento de idiomas, por ejemplo, si hacemos un análisis de los n-grams más comunes entre distintos idiomas podemos llegar a identificar diferencias entre los mismos. También podemos llegar a entrenar un modelo de clasificación a partir de los N-grams para una clasificación automática de idiomas (como lo veremos en la siguiente unidad).

<img src="https://drive.google.com/uc?export=view&id=1KkPaATcdr_XPVEbxppaE4DNAJ-S37TK3" width="100%">

Veamos un ejemplo con el conjunto de datos [Language Detection de Kaggle](https://www.kaggle.com/datasets/basilb2s/language-detection):

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/mindlab-unal/mlds4-datasets/main/u3/language.csv")
display(data.head())

Creamos una columna con el texto preprocesado:

In [ ]:
data = data.assign(
        text_prep = lambda df: df.text.apply(preprocess)
        )
display(data.head())

Vamos a calcular desde 2-grams hasta 4-grams de carácteres en los textos con el `CountVectorizer` para los textos en español:

In [ ]:
data_spa = (
        data
        .query("language == 'Spanish'")
        .text_prep
        )

vect_spa = CountVectorizer(
        ngram_range=(2, 4),
        analyzer="char"
        ).fit(data_spa)

Extraemos la representación de N-grams:

In [ ]:
counts = np.array(vect_spa.transform(data_spa).sum(axis=0)).flatten()
vocab = vect_spa.get_feature_names_out()
counts_spa = {word: count for word, count in zip(vocab, counts)}
display(counts_spa)

Ahora, repetimos el mismo proceso para textos en árabe:

In [ ]:
data_ara = (
        data
        .query("language == 'Arabic'")
        .text_prep
        )

vect_ara = CountVectorizer(
        ngram_range=(2, 4),
        analyzer="char"
        ).fit(data_ara)

Extraemos la representación de N-grams:

In [ ]:
counts = np.array(vect_ara.transform(data_ara).sum(axis=0)).flatten()
vocab = vect_spa.get_feature_names_out()
counts_ara = {word: count for word, count in zip(vocab, counts)}
display(counts_ara)

Veamos una comparativa de ambos idiomas usando nubes de palabras:

In [ ]:
!pip install wordcloud

Importamos la librería:

In [ ]:
from wordcloud import WordCloud

Creamos las nubes de palabras para ambos idiomas:

In [ ]:
wc_spa = (
        WordCloud(background_color="#FFFFFF")
        .generate_from_frequencies(counts_spa)
        )
wc_ara = (
        WordCloud(background_color="#FFFFFF")
        .generate_from_frequencies(counts_ara)
        )

Finalmente, mostramos las dos nubes de palabras:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(wc_spa)
ax[0].axis("off")
ax[0].set_title("Español")

ax[1].imshow(wc_ara)
ax[1].axis("off")
ax[1].set_title("Árabe")
fig.show()

Como se puede observar, en español predominan palabras de una única letra como `"a"`, `"e"`, `"o"` mientras que en árabe tenemos más 4-grams como `"tund"`, `"humi"` o `"acas"`.

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [Language model](https://en.wikipedia.org/wiki/Language_model).
- [Markov chain](https://en.wikipedia.org/wiki/Markov_chain).
- [N-grams](https://es.wikipedia.org/wiki/N-grama).
- _Fuente de los íconos_
    - Flaticon. Scrabble free icon [PNG]. https://www.flaticon.com/free-icon/scrabble_3367498
    - Flaticon. Icon Pack: A to Z Capital Letter | Flat [Pack Icon - PNG]. https://www.flaticon.com/packs/a-to-z-capital-letter-1

## Créditos
---

* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*